In [1]:
# pythonで絵文字を出力😯
print('\U0001F600')

😀


In [1]:
import os
import pytorch_lightning as pl
import torch
print(f'pytorch: {torch.__version__}')
print(f'cuda: {torch.cuda.is_available()}')
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import numpy as np

from src import lightning_model
from src.data import get_dataloaders

# Fix seed
pl.seed_everything(0, workers=True)


DATA_PATH = 'Flickr8k'
IMAGE_ENCODER_NAME = 'resnet18'
TEXT_ENCODER_NAME = 'albert-base-v2'

/root/anaconda3/envs/clip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch: 2.1.2+cu121
cuda: True


Seed set to 0


In [2]:
if not os.path.exists(DATA_PATH):
    print('Downloading the dataset from Kaggle...')
    #os.system('wget

train_loader, val_loader = get_dataloaders(data_dir=DATA_PATH, tokenizer_name=TEXT_ENCODER_NAME, batch_size=16, num_workers=2)

In [3]:
model = lightning_model.CLIPModel(image_encoder_alias=IMAGE_ENCODER_NAME, text_encoder_alias=TEXT_ENCODER_NAME)
trainer = pl.Trainer(max_epochs=10, enable_checkpointing=True, callbacks=[lightning_model.OverrideEpochStepCallback(), EarlyStopping(monitor='val/loss', patience=30, verbose=False)])
trainer.fit(model, train_loader, val_loader)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name             | Type           | Params
----------------------------------------------------
0 | image_encoder    | ImageEncoder   | 11.2 M
1 | text_

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


/root/anaconda3/envs/clip/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/root/anaconda3/envs/clip/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: You called `self.log('step', ...)` in your `on_validation_epoch_end` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don't intend to reduce the value (for instance when logging the global step or epoch) then you can use `self.logger.log_metrics({'step': ...})` instead.


Epoch 0:   3%|▎         | 60/2023 [01:39<54:23,  0.60it/s, v_num=3, train/loss_step=2.770]  

In [ ]:
from torchinfo import summary
summary(model, dtypes=[torch.bfloat16], input_size=(next(iter(train_loader))[0].shape[0], next(iter(train_loader))[0].shape[1], next(iter(train_loader))[0].shape[2], next(iter(train_loader))[0].shape[3]))

/root/anaconda3/envs/deeplearning/lib/python3.10/site-packages/torchinfo/torchinfo.py:216: UserWarning: Half precision is not supported with input_size parameter, and may output incorrect results. Try passing input_data directly.
  validate_user_params(


Layer (type:depth-idx)                             Output Shape              Param #
BaseModel                                          [256, 7]                  --
├─ViT: 1-1                                         [256, 7]                  --
│    └─PatchEmbedding: 2-1                         [256, 64, 128]            8,192
│    │    └─Conv2d: 3-1                            [256, 128, 8, 8]          4,736
│    └─CustomTransformerEncoder: 2-2               [256, 64, 128]            --
│    │    └─ModuleList: 3-2                        --                        1,851,392
│    └─Sequential: 2-3                             [256, 7]                  --
│    │    └─LayerNorm: 3-3                         [256, 128]                256
│    │    └─Linear: 3-4                            [256, 128]                16,512
│    │    └─GEGLU: 3-5                             [256, 64]                 --
│    │    └─Linear: 3-6                            [256, 7]                  455
Total params: 1,

In [ ]:
%load_ext tensorboard
%tensorboard  --logdir lightning_logs/